In [ ]:
import os
import numpy as np
import ceruleanml.data as data
import pandas as pd
from icevision.parsers import COCOMaskParser
from icevision.data import SingleSplitSplitter

In [ ]:
parser = COCOMaskParser(annotations_filepath="/root/data/tiled-cerulean-v2-partial-no-context/instances_Tiled Cerulean Dataset V2 No Context Files.json", img_dir="/root/data/tiled-cerulean-v2-partial-no-context/tiled_images_no_context/")

train_records = parser.parse(data_splitter=SingleSplitSplitter(), autofix=False)

In [ ]:
from ceruleanml.coco_stats import region_props_for_instance_type
coco_class_list = ["infra_slick", "natural_seep", "coincident_vessel", "recent_vessel", "old_vessel", "ambiguous"]

cat_tables = []
for cat in coco_class_list:
    cat_table = region_props_for_instance_type(train_records[0], cat)
    cat_tables.append(cat_table)
instance_stats_df_tiled = pd.concat(cat_tables)
    

In [ ]:
def print_table(df, name):
    print(df.to_markdown())
    df.to_csv(name)

In [ ]:
mean_summary = instance_stats_df_tiled.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3", "img_name"]).groupby("category").mean().round()
print_table(mean_summary, "mean_tiled.csv")

In [ ]:
max_summary = instance_stats_df_tiled.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3"]).groupby("category").max().round()
print_table(max_summary, "max_tiled.csv")

In [ ]:
min_summary = instance_stats_df_tiled.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3"]).groupby("category").min().round()

print_table(min_summary, "min_tiled.csv")

In [ ]:
count_summary = instance_stats_df_tiled.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3"]).groupby("category").count()
print_table(count_summary.rename(columns={'area':'count'}).iloc[:,0], "count_tiled.csv")

### Whole image dataset stats

In [ ]:
import os
import numpy as np
import ceruleanml.data as data
import pandas as pd
from icevision.parsers import COCOMaskParser
from icevision.data import SingleSplitSplitter

In [ ]:
parser = COCOMaskParser(annotations_filepath="/root/data/cv2-no-tiles-partial/instances_UntiledCeruleanDatasetV2NoContextFiles.json", img_dir="")

train_records = parser.parse(data_splitter=SingleSplitSplitter(),  autofix=False)

In [ ]:
from pathlib import Path
allpths = []
for d in train_records[0]:
    pths = list(
        Path(os.path.dirname(d.as_dict()["common"]["filepath"])).glob("*_*")
    )
    allpths.extend(pths)

In [ ]:
for pth in allpths:
    if (len(os.path.basename(str(pth)).split("_")) != 3
        and len(os.path.basename(str(pth)).split("_")) != 2):
        print(pth)

In [ ]:
from ceruleanml.coco_stats import region_props_for_instance_type_whole_image
from tqdm import tqdm
coco_class_list = ["infra_slick", "natural_seep", "coincident_vessel", "recent_vessel", "old_vessel", "ambiguous"]

cat_tables = []
for cat in tqdm(coco_class_list):
    cat_table = region_props_for_instance_type_whole_image(train_records[0], cat)
    cat_tables.append(cat_table)
instance_stats_df_no_tile = pd.concat(cat_tables)
    

In [ ]:
mean_summarynt = instance_stats_df_no_tile.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3", "img_name"]).groupby("category").mean().round()
print(mean_summarynt.to_markdown(), "mean_notile.csv")

In [ ]:
max_summarynt = instance_stats_df_no_tile.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3"]).groupby("category").max().round()
print_table(max_summarynt, "max_notile.csv")

In [ ]:
min_summarynt = instance_stats_df_no_tile.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3"]).groupby("category").min().round()

print_table(min_summarynt, "min_notile.csv")

In [ ]:
count_summarynt = instance_stats_df_no_tile.drop(columns = ["bbox-0","bbox-1","bbox-2","bbox-3"]).groupby("category").count()
print_table(count_summarynt.rename(columns={'area':'count'}).iloc[:,0], "count_notile.csv")

In [ ]:
print_table(instance_stats_df_no_tile, "all_notile.csv")

In [ ]:
print_table(instance_stats_df_tiled, "all_tiled.csv")